In [1]:
import os
import time
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython import display
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

W1010 13:03:56.595734 139930587498304 __init__.py:308] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W1010 13:03:56.613404 139930587498304 __init__.py:335] Limited tf.summary API due to missing TensorBoard installation.


In [2]:
print("GPU Available: ", tf.test.is_gpu_available())
print(tf.__version__)

GPU Available:  True
2.0.0-beta1


In [3]:
# warnings.filterwarnings('ignore')
tf.keras.backend.clear_session()  # For easy reset of notebook state.
np.set_printoptions(suppress=True, linewidth=120, precision=2)

In [4]:
def perf_measure(y_true, y_pred):
    
    cnf_matrix = confusion_matrix(y_true, y_pred)
    
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    
    FSCORE = np.divide((2*PPV*TPR), (PPV+TPR))
    
    return PPV, TPR, FSCORE, FNR, FPR, TNR

In [5]:
name_of_particle = 'Egammas'

X_train = np.load("matrices/" + name_of_particle +"_train.npy",)
y_train = np.load("matrices/" + name_of_particle +"_y_train.npy",)
X_val = np.load("matrices/" + name_of_particle +"_val.npy",)
y_val = np.load("matrices/" + name_of_particle +"_y_val.npy",)
X_test = np.load("matrices/" + name_of_particle +"_test.npy",)
y_test = np.load("matrices/" + name_of_particle +"_y_test.npy",)
X_train = X_train[:, :-3]
X_val = X_val[:, :-3]
X_test = X_test[:, :-3]
_, V = X_train.shape
K = 1
V

211

In [6]:
X_train_pos = X_train[np.where(y_train==0)]
X_train_neg = X_train[np.where(y_train==1)]
y_train_pos = y_train[np.where(y_train==0)]
y_train_neg = y_train[np.where(y_train==1)]

X_val_pos = X_val[np.where(y_val==0)]
X_val_neg = X_val[np.where(y_val==1)]
y_val_pos = y_val[np.where(y_val==0)]
y_val_neg = y_val[np.where(y_val==1)]

X_test_pos = X_test[np.where(y_test==0)]
X_test_neg = X_test[np.where(y_test==1)]
y_test_pos = y_test[np.where(y_test==0)]
y_test_neg = y_test[np.where(y_test==1)]

X_train_pos.shape[0] + X_train_neg.shape[0] == X_train.shape[0]

True

In [7]:
batch_size = 512

test_ds_pos = tf.data.Dataset.from_tensor_slices((X_test_pos, y_test_pos)).batch(batch_size) #.shuffle(1000)
test_ds_neg = tf.data.Dataset.from_tensor_slices((X_test_neg, y_test_neg)).batch(batch_size)#.shuffle(1000)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)
    
train_ds_pos = tf.data.Dataset.from_tensor_slices((X_train_pos, y_train_pos)).batch(batch_size)#.shuffle(1000)
train_ds_neg = tf.data.Dataset.from_tensor_slices((X_train_neg, y_train_neg)).batch(batch_size)#.shuffle(1000)
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)#.shuffle(1000)

val_ds_pos = tf.data.Dataset.from_tensor_slices((X_val_pos, y_val_pos)).batch(batch_size)#.shuffle(1000)
val_ds_neg = tf.data.Dataset.from_tensor_slices((X_val_neg, y_val_neg)).batch(batch_size)#.shuffle(1000)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)#.shuffle(1000)


### Encoder 

In [8]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, original_dim, intermediate_dim):
        super(Encoder, self).__init__()
        self.h1 = tf.keras.layers.Dense(units=original_dim, activation=tf.nn.relu, input_shape=(original_dim,))
        self.output_layer = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
    
    def call(self, x):
        h1 = self.h1(x)
        return self.output_layer(h1)


### Decoder 

In [9]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, intermediate_dim, original_dim):
        super(Decoder, self).__init__()
        self.h1 = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu, input_shape=(intermediate_dim,))
        self.output_layer = tf.keras.layers.Dense(units=original_dim, activation=tf.nn.relu)
    
    def call(self, z):
        h1 = self.h1(z)
        return self.output_layer(h1)
    

### Model

In [10]:
class AutoEncoder(tf.keras.Model):
    def __init__(self, intermediate_dim, original_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = Encoder(original_dim=original_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(intermediate_dim=intermediate_dim, original_dim=original_dim)
        
    def call(self, x):
        z = self.encoder(x)
        reconstructed = self.decoder(z)
        return reconstructed

### Loss Function

In [11]:
def loss(model, original):
    reconstruction_error = tf.reduce_mean(tf.square(model(original), original))
    return reconstruction_error

### Training

In [12]:
def train(loss, model, opt, original):
    with tf.GradientTape() as tape:
        gradients = tape.gradient(loss(model, original), model.trainable_variables)
        opt.apply_gradients(zip(gradients, model.trainable_variables))

### Instantiation

In [13]:
n_epochs = 200
autoencoder = AutoEncoder(intermediate_dim=100, original_dim=V)
opt = tf.optimizers.Adam(learning_rate=1e-6)


for epoch in range(n_epochs):
    for X_tr, y_tr in train_ds:
        train(loss, autoencoder, opt, X_tr)
        loss_values = loss(autoencoder, X_tr)
        original = tf.reshape(X_tr, (X_tr.shape[0], V, 1))
        reconstructed = tf.reshape(autoencoder(tf.constant(X_tr)), (X_tr.shape[0], V, 1))

    if epoch%5 == 0:
        print("epoch:", epoch)

autoencoder.save_weights("NN-ckecks/AutoEncoder"+ name_of_particle +".h5")
print("done!")

epoch: 0
epoch: 5
epoch: 10
epoch: 15
epoch: 20
epoch: 25
epoch: 30
epoch: 35
epoch: 40
epoch: 45
epoch: 50
epoch: 55
epoch: 60
epoch: 65
epoch: 70
epoch: 75
epoch: 80
epoch: 85
epoch: 90
epoch: 95
epoch: 100
epoch: 105
epoch: 110
epoch: 115
epoch: 120
epoch: 125
epoch: 130
epoch: 135
epoch: 140
epoch: 145
epoch: 150
epoch: 155
epoch: 160
epoch: 165
epoch: 170
epoch: 175
epoch: 180
epoch: 185
epoch: 190
epoch: 195
done!


In [14]:
autoencoder.summary()

Model: "auto_encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Encoder)            multiple                  65932     
_________________________________________________________________
decoder (Decoder)            multiple                  31411     
Total params: 97,343
Trainable params: 97,343
Non-trainable params: 0
_________________________________________________________________


In [15]:
new_autoencoder = AutoEncoder(intermediate_dim=100, original_dim=V)
new_autoencoder.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6))

# Since In this implementation instead of weight we are dealing 
# with codes and classes therefore the traditional serialization and
# deserialization is not possible. So we have to first initialze
# the model (which is code) and then load the weights 
# Ref: https://colab.research.google.com/drive/172D4jishSgE3N7AO6U2OKAA_0wNnrMOq#scrollTo=OOSGiSkHTERy

cntr = 0
for i, j in train_ds_pos:
    if cntr == 0:
        new_autoencoder.train_on_batch(i[:1], j[:1])
    cntr += 1 

new_autoencoder.load_weights("NN-ckecks/AutoEncoder"+ name_of_particle +".h5")
test_predictions = new_autoencoder.predict(X_test)
probabilities = tf.nn.sigmoid(test_predictions)
labels_pred = tf.argmax(probabilities, axis=1)


labels_true = []
for i, j in test_ds:
    for k in j.numpy():
        labels_true.append(int(k))


W1010 13:09:21.601952 139930587498304 deprecation.py:323] From /home/Soroosh/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
mse = np.mean(np.power(X_test - test_predictions, 2), axis=1)
df_error = pd.DataFrame({'reconstruction_error': mse, 'y_test': y_test},)
# df_error.describe()

In [17]:
df_error.describe()

,reconstruction_error,y_test
count,1.781300e+04,17813.000000
mean,3.816610e+06,0.015943
std,5.891831e+06,0.125260
min,4.446918e+03,0.000000
25%,1.964878e+06,0.000000
50%,3.003276e+06,0.000000
75%,5.158618e+06,0.000000
max,7.112221e+08,1.000000


In [18]:
mse

array([3404320.36, 1021893.42, 3587801.42, ..., 3818430.85, 2203437.18, 4414597.13])

In [19]:
y_pred = (df_error.reconstruction_error > 0.5).tolist()
y_pred = [1 if i == True else 0 for i in y_pred ]

In [20]:
FSCORE = precision_recall_fscore_support(y_test, y_pred, average='weighted')
PPV, TPR, FSCORE, FNR, FPR, TNR = perf_measure(y_true=y_test, y_pred=y_pred)

PPV, TPR, FSCORE, FNR, FPR, TNR

/home/Soroosh/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide
/usr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide


(array([ nan, 0.02]),
 array([0., 1.]),
 array([ nan, 0.03]),
 array([1., 0.]),
 array([0., 1.]),
 array([1., 0.]))

In [21]:
name_of_particle

'Egammas'

In [22]:
(1, ) * (2, )

TypeError: can't multiply sequence by non-int of type 'tuple'